# This notebook serves to create a model using SGD model for our TFW data

In [ ]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
# from pandas_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor

import warnings
import mlflow
from modeling.config import EXPERIMENT_NAME
TRACKING_URI = open("../.mlflow_uri").read().strip()

ROOT = os.environ.get('PWD')

warnings.filterwarnings('ignore')






# Data cleaning and feature engineering has taken place to run the baseline model, this data set will be used as it is for running the top secret model

In [ ]:
# set global default to be able to see all columns

pd.set_option('display.max_rows', None)

In [ ]:
# read data from csv: this is the time normed, filtered set with agg price data for the years in question

master_dummies = pd.read_csv('../data/master_with_dummies_1.csv')

In [ ]:
# create X and Y sets for train test split

X = master_dummies.drop(['listing_id','inquiry_count'],axis=1)
Y = master_dummies['inquiry_count']

In [ ]:
# train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [ ]:
# we love to normalize our data!!

from sklearn.preprocessing import MinMaxScaler


In [ ]:
the_list = ['mean_price_per_day','adult_count','children_count','pets_count','length_stay','bathrooms','bedrooms','max_guests','living_area']

In [ ]:
# Scaling with MinMaxScaler
scaler_norm = MinMaxScaler()
X_train_scaled_norm = scaler_norm.fit_transform(X_train[the_list])
X_test_scaled_norm = scaler_norm.transform(X_test[the_list])

In [ ]:
# Concatenating normalized columns 
X_train_preprocessed_norm = np.concatenate([X_train_scaled_norm, X_train.drop(the_list, axis=1)], axis=1)
X_test_preprocessed_norm = np.concatenate([X_test_scaled_norm, X_test.drop(the_list, axis=1)], axis=1)

In [ ]:
X_train_preprocessed_norm.shape

In [ ]:
X_test_preprocessed_norm.shape

## Trainining the model and tracking with MLFlow

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run()
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
# set the model to SDGRegessor
# 

reg = SGDRegressor()
reg.fit(X_train_preprocessed_norm, y_train)


In [ ]:
from sklearn.metrics import mean_squared_error

score = reg.score(X_train_preprocessed_norm, y_train)
y_pred_train = reg.predict(X_train_preprocessed_norm)
y_pred_test = reg.predict(X_test_preprocessed_norm)
 
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)

print("R-squared:", score)
print("RMSE train:", rmse_train)
print("RMSE test:", rmse_test)

In [ ]:
params = {
      "model":"SGD",
      "year":'2019',
      "train_test_split": 30,
      "normalized data": 'yes',
      "metric": 'rmse and R2'
  }

In [ ]:
mlflow.log_params(params)
mlflow.set_tag("running_from_jupyter", "SVR model 2019")
mlflow.log_metric("train -" + "RMSE", rmse_train)
mlflow.log_metric("test -" + "RMSE", rmse_test)
# mlflow.log_artifact("../models")
# mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
mlflow.get_run(run_id=run.info.run_id)

## Checking the experiments

while the next cell is running you will not be able to run other cells in the notebook

In [ ]:
!mlflow ui